# client端

In [5]:
import numpy
from concrete import fhe
from concrete.ml.quantization import QuantizedArray
from concrete.ml.quantization.quantizers import(
    QuantizedArray,
    MinMaxQuantizationStats,
)
from concrete.ml.quantization.quantized_ops import (
    QuantizationOptions,
)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# 加载Iris数据集
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 将输出标签进行独热编码
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))

# 数据标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)


In [6]:
# 定义一个类来表示NumPy版的FFNN模型
class NumPyFFNN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.weights1 = np.random.randn(input_dim, hidden_dim)
        self.bias1 = np.zeros(hidden_dim)
        self.weights2 = np.random.randn(hidden_dim, output_dim)
        self.bias2 = np.zeros(output_dim)
        self.relu_circuit=None

    def q_relu(self, n_bits:int,input:numpy.ndarray):
        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        stats=MinMaxQuantizationStats(n_bits)
        stats.compute_quantization_stats(input)
        # Quantize the inputs with n_bits
        q_inputs = QuantizedArray(n_bits, input,options=options,stats=stats)
        @fhe.compiler({"q_inputs":"encrypted"})
        def q_relu_impl(q_inputs):
            return np.maximum(q_inputs, 0)
        inputset=[np.random.randint(-128, 128, size=input.size)
                  for i in range(10000)]
        circuit=q_relu_impl.compile(inputset)
        circuit.keygen()
        self.relu_circuit=circuit
        return circuit

    def q_sub(self, n_bits:int,input_0 :numpy.ndarray, input_1:numpy.ndarray):

        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        stats=MinMaxQuantizationStats(n_bits)
        stats.compute_quantization_stats(np.hstack((input_0, input_1)))
        # Quantize the inputs with n_bits
        q_inputs_0 = QuantizedArray(n_bits, input_0,options=options,stats=stats)
        q_inputs_1 = QuantizedArray(n_bits, input_1,options=options,stats=stats)
        @fhe.compiler({"q_inputs_0":"encrypted","q_inputs_1":"encrypted"})
        def q_sub_impl(q_inputs_0,q_inputs_1):
            return q_inputs_0-q_inputs_1
        inputset=[(np.random.randint(-128, 128, size=input_0.size),np.random.randint(-128, 128, size=input_1.size))
                  for i in range(10000)]
        circuit=q_sub_impl.compile(inputset)
        q_result=circuit.encrypt_run_decrypt(q_inputs_0.qvalues,q_inputs_1.qvalues)
        return q_result*q_inputs_0.quantizer.scale

    def q_add(self, n_bits:int,input_0 :numpy.ndarray, input_1:numpy.ndarray):

        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        stats=MinMaxQuantizationStats(n_bits)
        stats.compute_quantization_stats(np.hstack((input_0, input_1)))
        # Quantize the inputs with n_bits
        q_inputs_0 = QuantizedArray(n_bits, input_0,options=options,stats=stats)
        q_inputs_1 = QuantizedArray(n_bits, input_1,options=options,stats=stats)
        @fhe.compiler({"q_inputs_0":"encrypted","q_inputs_1":"encrypted"})
        def q_add_impl(q_inputs_0,q_inputs_1):
            return q_inputs_0+q_inputs_1
        inputset=[(np.random.randint(-128, 128, size=input_0.size),np.random.randint(-128, 128, size=input_1.size))
                  for i in range(10000)]
        circuit=q_add_impl.compile(inputset)
        q_result=circuit.encrypt_run_decrypt(q_inputs_0.qvalues,q_inputs_1.qvalues)
        return q_result*q_inputs_0.quantizer.scale

    def q_mul(self, n_bits:int,input_0 :numpy.ndarray, input_1:numpy.ndarray):
        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        q_inputs_0 = QuantizedArray(n_bits, input_0, is_signed=True,options=options)
        q_inputs_1 = QuantizedArray(n_bits, input_1, is_signed=True,options=options)
        @fhe.compiler({"q_inputs_0":"encrypted","q_inputs_1":"encrypted"})
        def q_mul_impl(q_inputs_0,q_inputs_1):
            return q_inputs_0*q_inputs_1
        inputset=[(np.random.randint(-128, 128, size=input_0.size),np.random.randint(-128, 128, size=input_1.size))
                  for i in range(10000)]
        circuit=q_mul_impl.compile(inputset)
        q_result=circuit.encrypt_run_decrypt(q_inputs_0.qvalues,q_inputs_1.qvalues)
        return q_result*q_inputs_0.quantizer.scale*q_inputs_1.quantizer.scale

    def q_div(self, n_bits:int,input_0 :numpy.ndarray, input_1:numpy.ndarray):
        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        q_inputs_0 = QuantizedArray(n_bits, input_0, is_signed=True,options=options)
        q_inputs_1 = QuantizedArray(n_bits, input_1, is_signed=True,options=options)
        @fhe.compiler({"q_inputs_0":"encrypted"})
        def q_div_impl(q_inputs_0):
            return numpy.floor_divide(q_inputs_0,q_inputs_1.qvalues)
        inputset=[np.random.randint(-128, 128, size=input_0.size)
                  for i in range(10000)]
        circuit=q_div_impl.compile(inputset)
        print(circuit)
        q_result=circuit.encrypt_run_decrypt(q_inputs_0.qvalues)
        print(q_result)
        return q_result*q_inputs_0.quantizer.scale/q_inputs_1.quantizer.scale

    def q_matmul_(
        self,
        n_bits: int,
        inputs: numpy.ndarray,
        weights: numpy.ndarray,
    ):
        options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
        q_inputs = QuantizedArray(n_bits, inputs,is_signed=True,options=options)
        q_weights = QuantizedArray(n_bits, weights,is_signed=True,options=options)

        @fhe.compiler({"q_inputs":"encrypted","q_weights":"clear"})
        def q_matmul_impl(q_inputs,q_weights):
            return q_inputs@q_weights
        inputset=[(np.random.randint(-128, 128, size=inputs.shape),np.random.randint(-128, 128, size=weights.shape))
                  for i in range(10000)]
        circuit=q_matmul_impl.compile(inputset)
        q_result=circuit.encrypt_run_decrypt(q_inputs.qvalues,q_weights.qvalues)
        return q_result*q_inputs.quantizer.scale*q_weights.quantizer.scale



    def forward(self, x):
        out1 = np.dot(x, self.weights1) + self.bias1
        # out1 = np.maximum(out1, 0)  # ReLU activation function
        out1[out1 < 0] = 0
        out2 = np.dot(out1, self.weights2) + self.bias2
        return out2

    def q_forward(self, x):
        n_bits=8
        # 线性层
        out1 = self.q_matmul_(n_bits,x,self.weights1)+self.bias1
        # 激活层
        # 执行激活函数需要判断tlu是否存在
        if self.relu_circuit is None:
            out1_relu = self.q_relu(n_bits,out1)  # ReLU activation function
        else:
            out1_relu=self.relu_circuit.encrypt_run_decrypt(out1)

        out2=self.q_matmul_(n_bits,out1_relu, self.weights2)
        return out2

    def backward(self, x, y, learning_rate):
        # 前向传播
        out1 = np.dot(x, self.weights1) + self.bias1
        out1 = np.maximum(out1, 0)  # ReLU activation function
        out2 = np.dot(out1, self.weights2) + self.bias2
        loss = np.mean((out2 - y) ** 2)  # 均方误差损失

        # 反向传播
        delta_out2 = 2 * (out2 - y) / len(x)
        delta_weights2 = np.dot(out1.T, delta_out2)
        delta_bias2 = np.sum(delta_out2, axis=0)
        delta_out1 = np.dot(delta_out2, self.weights2.T)
        # delta_out1[out1 <= 0] = 0  # ReLU反向传播
        delta_relu_out = np.where(out1 <= 0, 0, delta_out1) #不使用密文，直接算
        delta_weights1 = np.dot(x.T, delta_relu_out)
        delta_bias1 = np.sum(delta_relu_out, axis=0)

        # 更新参数
        self.weights2 -= learning_rate * delta_weights2
        self.bias2 -= learning_rate * delta_bias2
        self.weights1 -= learning_rate * delta_weights1
        self.bias1 -= learning_rate * delta_bias1

        return loss

    def q_backward(self, x, y, learning_rate):
        n_bits = 8
        # 前向传播
        out1 = self.q_matmul_(n_bits, x, self.weights1)+self.bias1
        # 执行激活函数需要判断tlu是否存在
        if self.relu_circuit is None:
            out1_relu = self.q_relu(n_bits,out1)  # ReLU activation function
        else:
            out1_relu=self.relu_circuit.encrypt_run_decrypt(out1)

        out2 = self.q_matmul_(n_bits, out1_relu, self.weights2)+ self.bias2
        loss = np.mean((out2 - y) ** 2)  # 均方误差损失

        # 反向传播
        delta_out2=self.q_div(n_bits,self.q_mul(n_bits,self.q_sub(n_bits,out2,y),2),len(x))
        delta_weights2=self.q_matmul_(n_bits,out1_relu.T,delta_out2)  # 密文和密文矩阵乘法不能超过16位
        delta_bias2 = np.sum(delta_out2, axis=0) #delta_out2是密文，所以delta_bias2是密文
        delta_out1=self.q_matmul_(n_bits,delta_out2,self.weights2.T)
        delta_out1_sigmoid=self.q_mul(n_bits,self.q_mul(n_bits,delta_out1,out1_relu),self.q_sub(n_bits,1,out1_relu))
        delta_weights1=self.q_mul(n_bits,x.T,delta_out1_sigmoid)
        delta_bias1 = np.sum(delta_out1_sigmoid, axis=0)

        # 更新参数
        self.weights2 -= learning_rate * delta_weights2
        self.bias2 -= learning_rate * delta_bias2
        self.weights1 -= learning_rate * delta_weights1
        self.bias1 -= learning_rate * delta_bias1

        return loss


# 模型参数
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = y_train.shape[1]


In [7]:
# 创建NumPy模型实例
numpy_model = NumPyFFNN(input_dim, hidden_dim, output_dim)

In [8]:
# 随机选择一批训练数据
batch_indices = np.random.choice(len(X_train), 32, replace=False)
x_batch = X_train[batch_indices]
y_batch = y_train[batch_indices]

In [9]:
#量化x_batch,和weights
n_bits=8
inputs=x_batch
weights=numpy_model.weights1
options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
q_inputs = QuantizedArray(n_bits, inputs,is_signed=True,options=options)
q_weights = QuantizedArray(n_bits, weights,is_signed=True,options=options)

In [10]:
# 创建电路
@fhe.compiler({"q_inputs":"encrypted","q_weights":"clear"})
def q_matmul_impl(q_inputs,q_weights):
    return q_inputs@q_weights
inputset=[(np.random.randint(-128, 128, size=inputs.shape),np.random.randint(-128, 128, size=weights.shape))
          for i in range(10000)]
circuit=q_matmul_impl.compile(inputset)

In [11]:
circuit.server.save("server.zip")

In [31]:
# Setting up a server
server = fhe.Server.load("server.zip")
print(server)

In [14]:
# Setting up clients
client=circuit.client
print(client)

In [15]:
# Generating keys (on the client)
client.keys.generate()

In [19]:
# After serialization, send the evaluation keys to the server.
serialized_evaluation_keys: bytes = client.evaluation_keys.serialize()

In [32]:
# The next step is to encrypt your inputs and request the server to perform some computation.
# This can be done in the following way:
arg: fhe.Value = client.encrypt(q_inputs.qvalues,q_weights.qvalues)
serialized_arg0: bytes = arg[0].serialize()
serialized_arg1: bytes = arg[1].serialize()

In [34]:
# Performing computation (on the server)
deserialized_evaluation_keys = fhe.EvaluationKeys.deserialize(serialized_evaluation_keys)
deserialized_arg0 = fhe.Value.deserialize(serialized_arg0)
deserialized_arg1 = fhe.Value.deserialize(serialized_arg1)
result: fhe.Value = server.run(deserialized_arg0,deserialized_arg1, evaluation_keys=deserialized_evaluation_keys)
serialized_result: bytes = result.serialize()

In [35]:
# Decrypting the result (on the client)
deserialized_result = fhe.Value.deserialize(serialized_result)
decrypted_result = client.decrypt(deserialized_result)
print(decrypted_result)

[[  674  5465  5215 ...   441 -3618  2834]
 [  148 -1090 -1508 ...  -413 -3053 -5110]
 [ 1997 -4304  1624 ...  2308 10210 13405]
 ...
 [  346  2329  1289 ...  -348 -5751 -4436]
 [ -696  4120   734 ... -1121 -3879 -3254]
 [ -580 -5161 -4189 ...   -62  3429 -1810]]


In [36]:
#反量化+bias（on the client）
out1=decrypted_result*q_inputs.quantizer.scale*q_weights.quantizer.scale+numpy_model.bias1

[[ 0.30637847  2.48421117  2.37056931 ...  0.20046425 -1.64462507
   1.28824418]
 [ 0.06727598 -0.49547853 -0.68548773 ... -0.18773636 -1.38779446
  -2.32283972]
 [ 0.90777122 -1.95645835  0.73821756 ...  1.0491417   4.64113377
   6.09347681]
 ...
 [ 0.15728034  1.05868762  0.58593746 ... -0.15818948 -2.61421746
  -2.01646125]
 [-0.31637895  1.87281794  0.33365252 ... -0.50957012 -1.76326718
  -1.47916252]
 [-0.26364913 -2.34602266 -1.90418309 ... -0.02818318  1.55871182
  -0.8227671 ]]


In [ ]:
#量化out
options=QuantizationOptions(n_bits,is_symmetric=True,is_signed=True)
q_out1 = QuantizedArray(n_bits, inputs,is_signed=True,options=options)

In [39]:
#编译电路
@fhe.compiler({"q_inputs":"encrypted"})
def q_relu_impl(q_inputs):
    return np.maximum(q_inputs, 0)
inputset=[np.random.randint(-128, 128, size=out1.size)for i in range(10000)]
circuit=q_relu_impl.compile(inputset)
circuit.server.save("server.zip")

In [40]:
# Setting up a server
server = fhe.Server.load("server.zip")

In [ ]:
# Setting up clients
client=circuit.client
# 生成密钥
client.keys.generate()
serialized_evaluation_keys: bytes = client.evaluation_keys.serialize()
# 加密输入数据
arg: fhe.Value = client.encrypt(q_out1.qvalues)
serialized_arg: bytes = arg.serialize()


In [ ]:
# Performing computation (on the server)
deserialized_evaluation_keys = fhe.EvaluationKeys.deserialize(serialized_evaluation_keys)
deserialized_arg = fhe.Value.deserialize(serialized_arg0)
result: fhe.Value = server.run(deserialized_arg0, evaluation_keys=deserialized_evaluation_keys)
serialized_result: bytes = result.serialize()

In [ ]:
# Decrypting the result (on the client)
deserialized_result = fhe.Value.deserialize(serialized_result)
decrypted_result = client.decrypt(deserialized_result)
print(decrypted_result)